In [128]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import os
project_path = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('.')), '../'))
import sys
sys.path.append(project_path)
from util.visualization import draw_lines
from util.visualization import draw_scatters
from util.evaluate_process import classifier_evaluate
from collections import Counter
from sklearn.metrics import auc, roc_curve

In [2]:
df = pd.read_csv('../../data/preprocessed.samecar.csv')

In [3]:
feats = [
    'colorp1', 'colorp2', 
    'fuel_typep1', 'fuel_typep2','displacement_standard1', 'displacement_standard2',
    'gearboxp1', 'gearboxp2', 'displacement_diff', 'displacement_diff_sparse',
    'mile_diff', 'mile_diff_sparse', 'mile_diff_rate', 'mile_diff_rate_sparse',
    'year_diff', 'year_diff_sparse', 'licensed_city_diff_sparse', 'title_diff', 
    'title_diff_sparse', 'register_time_diff', 'register_time_diff_sparse',
    'is_import_diff_sparse', 'transfer_times_diff', 'transfer_times_diff_sparse'
]
# 只保留离散特征
sparse_feats = [
    'colorp1', 'colorp2', 
    'fuel_typep1', 'fuel_typep2','displacement_standard1', 'displacement_standard2',
    'gearboxp1', 'gearboxp2', 'displacement_diff_sparse',
    'mile_diff_sparse', 'mile_diff_rate_sparse',
    'year_diff_sparse', 'licensed_city_diff_sparse', 
    'title_diff_sparse', 'register_time_diff_sparse',
    'is_import_diff_sparse', 'transfer_times_diff_sparse'
]

In [4]:
rdf = df[sparse_feats]
X_train, X_test, y_train, y_test = train_test_split(rdf, df.is_same, test_size=0.25, random_state=10)
len(X_train), len(X_test)

(8038, 2680)

In [70]:
# 计算信息熵
def cal_entropy(data):
    if len(data) == 0:
        return 0
    c = Counter(data)
    l = len(data)
    return -sum([c.get(k)/l * math.log2(c.get(k)/l) for k in c])
    

In [181]:
def gen_tree(df, depth=0, min_samples_split=6, max_depth=10, condition='-', min_gain_split=0.001):
    ori_entropy = cal_entropy(df.target)
    size = len(df)
    ret = {
            'ent': ori_entropy,
            'size': size,
            'tags': dict(Counter(df.target)),
            'depth': depth
    }
    if size < min_samples_split:
        # 这个节点数量太少, 不应该分裂
        ret['reason'] = 'less than min_samples_split'
        return ret
    if depth >= max_depth:
        # 太深了, 停止分裂
        ret['reason'] = 'to deep'
        return ret
    ###
    best_ent_gain = ori_entropy
    best_key = None
    ###
    for k in df.columns:
        if k == 'target':
            continue
        sum_ent = sum([cal_entropy(df[df[k] == v].target) for v in set(df[k])])
        ent_gain = ori_entropy - sum_ent
        if best_key is None or ent_gain > best_ent_gain:
            best_key = k
            best_ent_gain = ent_gain
    ###
    ret['ent_gain'] = best_ent_gain
    ret['key'] = best_key
    ###
    if best_ent_gain ==0: #< min_gain_split:
        # 提升太小, 停止分裂
        # 这里应该是小于 min_gain_split, 但是id 这个算法很快就停止了, 效果非常差
        # 这里可以牺牲gain, 来换来效果提升
        ret['reason'] = 'gain too small'
        return ret
    key_counter = dict(Counter(df[best_key]))
    ret['leaf num'] = key_counter
    ret['trees'] = {}
    ###
    for v in key_counter.keys():
        ret['trees'][v] = gen_tree(df[df[best_key] == v].drop(best_key, axis=1), depth=depth+1, condition=f'{best_key}={v}')
    return ret

In [182]:
rdf = X_train.copy()
rdf['target'] = y_train
adt = gen_tree(rdf)
adt

{'ent': 0.9185030429380243,
 'size': 8038,
 'tags': {0: 5357, 1: 2681},
 'depth': 0,
 'ent_gain': -0.17425739201674895,
 'key': 'gearboxp1',
 'leaf num': {-1: 5198, 1: 2840},
 'trees': {-1: {'ent': 0.38441612831796285,
   'size': 5198,
   'tags': {0: 4808, 1: 390},
   'depth': 1,
   'ent_gain': 0.0019677869967467676,
   'key': 'mile_diff_sparse',
   'leaf num': {1.0: 1604, 2.0: 320, 0.0: 3271, 0.5: 3},
   'trees': {1.0: {'ent': 0.2389664987991978,
     'size': 1604,
     'tags': {0: 1541, 1: 63},
     'depth': 2,
     'ent_gain': 0.0,
     'key': 'fuel_typep1',
     'reason': 'gain too small'},
    2.0: {'ent': 0.11611507530476972,
     'size': 320,
     'tags': {1: 315, 0: 5},
     'depth': 2,
     'ent_gain': 0.0,
     'key': 'fuel_typep1',
     'reason': 'gain too small'},
    0.0: {'ent': 0.027366767217248577,
     'size': 3271,
     'tags': {0: 3262, 1: 9},
     'depth': 2,
     'ent_gain': 0.0,
     'key': 'fuel_typep1',
     'reason': 'gain too small'},
    0.5: {'ent': -0.0,
  

In [193]:
def tree_to_dot(tree_dict):
    # node: 0 [label="abc"];
    # line: 0 -> 1;
    nodes = [];
    lines = [];
    index = [0];
    def _process(struct, parent_id=None, path=''):
        cnt = '\\n'.join([f'{key}:{struct.get(key, "-")}' for key in ['key', 'ent', 'ent_gain', 'size', 'tags', 'reason']])
        node = f'{index[0]} [label="{cnt}"];'
        nodes.append(node)
        if parent_id is not None:
            if path:
                lines.append(f'{parent_id} -> {index[0]} [headlabel="{path}"];')
            else:
                lines.append(f'{parent_id} -> {index[0]};')
        ###
        selfindex = index[0]
        index[0] += 1
        if 'trees' in struct:
            for v in struct['trees']:
                _process(struct['trees'][v], selfindex, path=v)
    ###
    _process(tree_dict)
    nodesstr = '\n'.join(nodes)
    linesstr = '\n'.join(lines)
    outputstr = 'digraph Tree {\nnode [shape=box] ;\n %s\n%s\n}' % (nodesstr, linesstr)
    return outputstr

## 这里dot 是一个绘图命令
## open 是mac 上的打开文件命令
def draw_tree(tree, save_path):
    s = tree_to_dot(tree)
    with open(f'{save_path}.dot', 'w') as f:
        f.write(s)
    os.system(f'dot -Tpng {save_path}.dot -o {save_path}.png && open {save_path}.png')

In [190]:
draw_tree(adt, 'id3_same_car')

In [131]:
## 测试集表现
pred, prob = predict(adt, X_test)
eval_ret = classifier_evaluate(y_test, pred)
print(eval_ret)

fpr, tpr, thresholds = roc_curve(y_test, prob)
roc_auc = auc(fpr, tpr)
print(f'auc{roc_auc}')
draw_lines(fpr, [tpr, fpr], ['tp', 'fp'])

{'recall': 0.9644495412844036, 'precision': 0.9428251121076233, 'accuracy': 0.9694029850746269, 'tp': 841, 'fp': 51, 'tn': 1757, 'fn': 31}
auc0.9689295029227897


In [135]:
# 训练集
pred, prob = predict(adt, X_train)
eval_ret = classifier_evaluate(y_train, pred)
print(eval_ret)

fpr, tpr, thresholds = roc_curve(y_train, prob)
roc_auc = auc(fpr, tpr)
print(f'auc{roc_auc}')
draw_lines(fpr, [tpr, fpr], ['tp', 'fp'])

{'recall': 0.9709063782170831, 'precision': 0.9397111913357401, 'accuracy': 0.9695197810400598, 'tp': 2603, 'fp': 167, 'tn': 5190, 'fn': 78}
auc0.9702748905331994


In [200]:
### 这个是按照cart 树的思路做的, id3 + 二叉树
def gen_tree_bi(df, depth=0, min_samples_split=6, min_samples_leaf=3, max_depth=10):
    ori_entropy = cal_entropy(df.target)
    size = len(df)
    ret = {
            'ent': ori_entropy,
            'size': size,
            'tags': dict(Counter(df.target)),
            'depth': depth
    }
    if size < min_samples_split:
        # 这个节点数量太少, 不应该分裂
        ret['reason'] = 'less than min_samples_split'
        return ret
    if depth >= max_depth:
        # 太深了, 停止分裂
        ret['reason'] = 'to deep'
        return ret
    ###
    best_ent_gain = ori_entropy + 1
    best_key = None
    best_val = None
    ###
    for k in df.columns:
        if k == 'target':
            continue
        uniq_val = set(df[k])
        for v in uniq_val:
            if len(df[df[k] <= v]) == size or len(df[df[k] <= v]) == 0:
                continue
            ent1 = cal_entropy(df[df[k] <= v].target)
            ent2 = cal_entropy(df[df[k] >  v].target)
            ent_gain = ori_entropy - ent1 - ent2
            if best_key is None or ent_gain > best_ent_gain:
                best_key = k
                best_val = v
                best_ent_gain = ent_gain
    ###
    ret['ent_gain'] = best_ent_gain
    ret['key'] = best_key
    ret['val'] = best_val
    df1 = df[df[best_key] <= best_val]
    df2 = df[df[best_key] >  best_val]
    ret['leaf num'] = [len(df1), len(df2)]
    if len(df1) < min_samples_leaf or len(df2) < min_samples_leaf:
        # 子节点数量过少, 不应该分裂
        ret['reason'] = 'leaf samples less than min_samples_leaf'
        return ret
    tree1 = gen_tree_bi(df1, depth=depth+1)
    tree2 = gen_tree_bi(df2, depth=depth+1)
    ret['tree1'] = tree1
    ret['tree2'] = tree2
    ret['trees'] = {}
    ret['trees'][f'{best_key} <= {best_val}'] = tree1
    ret['trees'][f'{best_key} > {best_val}'] = tree2
    return ret

In [157]:
def predict_process_bi(decision_tree, data):
    prob = decision_tree['tags'][1]/(decision_tree['tags'][1] + decision_tree['tags'][0])
    label = 1 if decision_tree['tags'][1] > decision_tree['tags'][0] else 0
    ####
    if 'tree1' in decision_tree:
        key = decision_tree['key']
        val = decision_tree['val']
        if data[key] <= val:
            tree = decision_tree['tree1']
        else:
            tree = decision_tree['tree2']
        ret = predict_process(tree, data)
        label = ret[0]
        prob = ret[1]
    return label, prob

    
def predict_bi(decision_tree, df):
    d11 = df.to_dict()
    size = len(df)
    d12 = [{k: d11[k][i] for k in d11} for i in df.index]
    preds = []
    probs = []
    for data in d12:
        ret = predict_process_bi(decision_tree, data)
        preds.append(ret[0])
        probs.append(ret[1])
    return preds, probs

In [201]:
rdf = X_train.copy()
rdf['target'] = y_train
id3_tree = gen_tree_bi(rdf)
## 测试集表现
pred, prob = predict_bi(id3_tree, X_test)
eval_ret = classifier_evaluate(y_test, pred)
print(eval_ret)

fpr, tpr, thresholds = roc_curve(y_test, prob)
roc_auc = auc(fpr, tpr)
print(f'auc{roc_auc}')
draw_lines(fpr, [tpr, fpr], ['tp', 'fp'])

{'recall': 0.9644495412844036, 'precision': 0.9428251121076233, 'accuracy': 0.9694029850746269, 'tp': 841, 'fp': 51, 'tn': 1757, 'fn': 31}
auc0.9689295029227897


In [202]:
draw_tree(id3_tree, 'id3_same_car_bi')

In [191]:
id3_tree

{'ent': 0.9185030429380243,
 'size': 8038,
 'tags': {0: 5357, 1: 2681},
 'depth': 0,
 'ent_gain': 0.47872276704003036,
 'key': 'register_time_diff_sparse',
 'val': 1.0,
 'leaf num': [5268, 2770],
 'tree1': {'ent': 0.11118995946796709,
  'size': 5268,
  'tags': {0: 5190, 1: 78},
  'depth': 1,
  'ent_gain': -1.702432016054134e-05,
  'key': 'is_import_diff_sparse',
  'val': 0.0,
  'leaf num': [1, 5267],
  'reason': 'leaf samples less than min_samples_leaf'},
 'tree2': {'ent': 0.3285903164300269,
  'size': 2770,
  'tags': {1: 2603, 0: 167},
  'depth': 1,
  'ent_gain': -0.1588886165694986,
  'key': 'fuel_typep1',
  'val': 4,
  'leaf num': [2726, 44],
  'tree1': {'ent': 0.33098787009382397,
   'size': 2726,
   'tags': {1: 2560, 0: 166},
   'depth': 2,
   'ent_gain': -0.17946684415675623,
   'key': 'displacement_standard1',
   'val': 4,
   'leaf num': [2650, 76],
   'tree1': {'ent': 0.3348896883930774,
    'size': 2650,
    'tags': {1: 2486, 0: 164},
    'depth': 3,
    'ent_gain': -9.1621616